In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import re

from sklearn.datasets import fetch_20newsgroups

In [2]:
# Provide list of categories to consider
categories = ['alt.atheism',
              'comp.graphics', 
              'comp.os.ms-windows.misc',
              'comp.sys.ibm.pc.hardware',  
              'comp.sys.mac.hardware',
              'comp.windows.x', 
              'misc.forsale', 
              'rec.autos', 
              'rec.motorcycles', 
              'rec.sport.baseball', 
              'rec.sport.hockey', 
              'sci.crypt', 
              'sci.electronics', 
              'sci.med', 
              'sci.space', 
              'soc.religion.christian', 
              'talk.politics.guns', 
              'talk.politics.mideast', 
              'talk.politics.misc', 
              'talk.religion.misc']

# Training subset
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
# Testing subset
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

print('Total samples in training data:',len(newsgroups_train.data))
print('Total samples in testing data:',len(newsgroups_test.data))

Total samples in training data: 11314
Total samples in testing data: 7532


In [3]:
glove_dir = './glove.6B.50d.txt'

vocab = {}
with open(glove_dir, encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        vocab[word] = vector
f.close()

print('Found %s word vectors.' %len(vocab))

Found 400000 word vectors.


In [18]:
def get_embeddings(text):
    embedding = np.zeros(50, dtype=float)
    i = 0
    for word in text.split(' '):
        if word in vocab:
            i += 1
            embedding += vocab[word]
    
    if i != 0:
        embedding /= i
        
    return embedding

In [5]:
def clean(text):
    """ Function to clean the text """
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    
    texter = re.sub(r'[^\w\s]', '', texter)
    
    # Remove numbers from string
    texter = re.sub(pattern=r"[+-]?\d+(?:\.\d+)?", repl="", string=texter, count=0, flags=0)
    texter = texter.replace("  ", " ")
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    if texter == "":
        texter = ""
    return texter

In [6]:
dict_categories = {0: 0,
                   1: 1, 
                   2: 1,
                   3: 1,  
                   4: 1,
                   5: 1,
                   6: 2, 
                   7: 3, 
                   8: 3, 
                   9: 3, 
                   10: 3,
                   11: 4, 
                   12: 4, 
                   13: 4, 
                   14: 4,
                   15: 5, 
                   16: 6,
                   17: 6, 
                   18: 6, 
                   19: 6}

def transform(df):
    X = []
    y = []

    texts = df.data
    categories = df.target

    for text in texts:
        text = clean(text)

        X.append(get_embeddings(text))

    for category in categories:
        index_y = dict_categories[category]
        y.append(index_y)

    return np.array(X), np.array(y)

In [7]:
# Hyperparameters
num_epochs = 10
batch_size = 128
dropout = 0.5
learning_rate = 0.001

In [8]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_dim, output_dim=20, dropout=0.5):
        super(DNN, self).__init__()
        self.hidden_layers = hidden_layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.hidden = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for i in range(hidden_layers)])
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        for layer in self.hidden:
            x = F.relu(layer(x))
        x = self.fc2(x)
#         x = F.softmax(self.fc2(x), dim=1)
        return x

In [10]:
# def build_random_dnn(input_dim, min_hidden_layer, max_hidden_layer, min_nodes, max_nodes, output_dim):

#     layer = list(range(min_hidden_layer,max_hidden_layer))
#     node = list(range(min_nodes, max_nodes))

#     num_layers = np.random.choice(layer)
#     num_nodes = np.random.choice(node)

#     dnn = DNN(input_dim, num_layers, num_nodes, output_dim)

#     return dnn

dnn = DNN(50, 1, 100, 7)
print(dnn)

DNN(
  (fc1): Linear(in_features=50, out_features=100, bias=True)
  (hidden): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
  )
  (fc2): Linear(in_features=100, out_features=7, bias=True)
)


In [11]:
# Loss function
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(dnn.parameters(), lr=learning_rate)

In [19]:
X_train, y_train = transform(newsgroups_train)

## create iterator objects for train datasets
X_tr = torch.tensor(X_train, dtype=torch.float)
y_tr = torch.tensor(y_train, dtype=torch.long)
train = TensorDataset(X_tr, y_tr)
trainloader = DataLoader(train, batch_size=batch_size)

In [20]:
# Training
for epoch in range(num_epochs):
    running_loss = 0
    for i, data in enumerate(trainloader):
        inputs, labels = data

        outputs = dnn(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

#         running_loss += loss.item()
        
        if (i+1) % 25 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.data))

Epoch [1/10], Step [25/88], Loss: 1.7797
Epoch [1/10], Step [50/88], Loss: 1.5987
Epoch [1/10], Step [75/88], Loss: 1.5440
Epoch [2/10], Step [25/88], Loss: 1.3736
Epoch [2/10], Step [50/88], Loss: 1.0655
Epoch [2/10], Step [75/88], Loss: 1.0724
Epoch [3/10], Step [25/88], Loss: 1.1595
Epoch [3/10], Step [50/88], Loss: 0.8829
Epoch [3/10], Step [75/88], Loss: 0.9191
Epoch [4/10], Step [25/88], Loss: 1.0528
Epoch [4/10], Step [50/88], Loss: 0.7909
Epoch [4/10], Step [75/88], Loss: 0.8276
Epoch [5/10], Step [25/88], Loss: 0.9536
Epoch [5/10], Step [50/88], Loss: 0.7443
Epoch [5/10], Step [75/88], Loss: 0.7822
Epoch [6/10], Step [25/88], Loss: 0.8925
Epoch [6/10], Step [50/88], Loss: 0.7259
Epoch [6/10], Step [75/88], Loss: 0.7634
Epoch [7/10], Step [25/88], Loss: 0.8575
Epoch [7/10], Step [50/88], Loss: 0.7144
Epoch [7/10], Step [75/88], Loss: 0.7529
Epoch [8/10], Step [25/88], Loss: 0.8339
Epoch [8/10], Step [50/88], Loss: 0.7060
Epoch [8/10], Step [75/88], Loss: 0.7462
Epoch [9/10], St

In [21]:
X_test, y_test = transform(newsgroups_test)

## create iterator objects for valid datasets
X_te = torch.tensor(X_test, dtype=torch.float)
y_te = torch.tensor(y_test, dtype=torch.long)
test = TensorDataset(X_te, y_te)
testloader = DataLoader(test)

In [22]:
total, correct = 0, 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        outputs = dnn(inputs)

        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        
        correct += (predicted == labels).sum()

print(f'Accuracy of the model is: {100*correct/total:.2f}%')

Accuracy of the model is: 74.02%
